## Tutorial de Theano: Básicos

Theano é uma biblioteca que adiciona um background matemático à Python, permitindo principalmente o cálculo de expressões matemáticas envolvendo matrizes e tensores. É um dos "concorrentes" ao Tensorflow


Prós:
* Mais simples 
* Mais aberto a hardware (libgpuarray como padrão)
* Integrado ao numpy
* Compilado (?)

Contras:
* Comunidade menor
* Não tem Sessions
* TensorBoard
* Compilado (?)

In [1]:
from theano import *
import theano.tensor as T
import numpy as np #np.arrays são usados para entrada e saída

Using gpu device 0: GeForce GT 630M (CNMeM is disabled)


### Theano
Workflow parecido com Tensorflow:

Descrevemos primeiro um grafo de computação

Estáticamente tipado: definimos o tipo das variáveis e elas não mudam

A função theano.function compila e prepara o grafo para uso, devolvendo uma função que pode ser chamada normalmente em python

In [2]:
#Faz um grafo de computação
x = T.dscalar()
y = T.dscalar()
z = x + y
f = function([x,y], z) #Compila em C

In [5]:
print(f)
print(z)
print(f(1, 2))
print(z.eval({x: 1, y: 2})) #Não precisa compilar, mas não tão flexível

Elemwise{add,no_inplace}.0
3.0
3.0


### Vetores e Matrizes
Exemplo: $Ax + b$

In [6]:
A = T.dmatrix('A')
x = T.dvector('x')
b = T.dvector('b')
z = theano.dot(A, x) + b 
f = function([A, x, b], z)

In [7]:
A = np.array([[1, -1], [-1, 1]])
x = np.array([2, 1])
b = np.array([1, 1])
f(A, x, b)

array([ 2.,  0.])

### Tipos de variáveis
Prefixo: tamanho e tipo
* b = byte
* w = inteiro de 16bits
* i = inteiro de 32bits
* l = inteiro de 64bits
* f = float
* d = double
* c = número complexo

Radical: forma

  scalar, vector, matrix, row, col, tensor3, tensor4, tensor5

### Variáveis com Estado

Variáveis compartilhadas são simbólicas (podem ser usadas para construir o grafo de computação) e não-simbólicas (podem ser variáveis normais usando get_value() e set_value()

In [8]:
from theano import shared
count = shared(0) #Cria um "0" compartilhável 
inc = T.iscalar()
result = count + inc
acc = function([inc], result)

In [9]:
print(acc(1))
print(count.get_value())

1
0


In [10]:
count.set_value(acc(1))
print(count.get_value())
count.set_value(acc(1))
print(count.get_value())
count.set_value(acc(10))
print(count.get_value())

1
2
12


Essa lógica de atualização pode ser incluída na própria função por meio do argumento opcional updates, que recebe uma lista de pares (variável compartilhada, expressão do update) e realiza o update depois da computação

In [12]:
count = shared(0)
inc = T.iscalar()
acc = function([inc], count + inc, updates=[(count, count + inc)])

In [13]:
print(acc(1))
print(acc(1))
print(acc(20))
print(count.get_value())

1
2
22
22


### Variáveis Aleatórias
Um tipo especial de variável compartilhada. Se usa geradores de número aleatórios ("streams") que são chamados quando necessários

In [14]:
from theano.tensor.shared_randomstreams import RandomStreams
srng = RandomStreams(seed=0) 
num = srng.uniform()
f = function([], num)
g = function([], num, no_default_updates=True)

In [15]:
print("Atualiza gerador")
for _ in range(3):
    print(f())
print("\nNão atualiza gerador")
for _ in range(3):
    print(g())

Atualiza gerador
0.48604732751846313
0.6857123374938965
0.9855760335922241

Não atualiza gerador
0.19559641182422638
0.19559641182422638
0.19559641182422638


### Gradientes e Derivação (Básico)

O formato de grafo torna especialmente fácil computar derivadas e gradientes pela regra da cadeia/backpropagation, dado que cada nó é uma operação simples que compõe depois com outras para criar uma expressão complexa (ver: http://cs231n.github.io/optimization-2/ )

Com isso a função "grad" do Theano calcula simbólicamente o gradiente ou derivada de uma função:

In [21]:
from theano import pp #pretty print
x = T.dscalar()
y = 3 * x ** -2 
dy = T.grad(y, x) #dy/dx
f = theano.function([x], dy)
pp(dy)

'(((fill((TensorConstant{3} * (<TensorType(float64, scalar)> ** TensorConstant{-2})), TensorConstant{1.0}) * TensorConstant{3}) * TensorConstant{-2}) * (<TensorType(float64, scalar)> ** (TensorConstant{-2} - TensorConstant{1})))'

In [20]:
print(f(3), f(0), f(1))

-0.2222222222222222 -inf -6.0
